### Load some data from public FTX endpoint

In [ ]:
import requests

In [ ]:
raw = requests.get("https://ftx.com/api//markets/BTC/USD/candles?resolution=60").json()

In [ ]:
raw['result'][:2]

### Initialise a DataCatalog

We'll just use the current directory

In [ ]:
from nautilus_trader.persistence.catalog import ParquetDataCatalog

catalog = ParquetDataCatalog("./")

In [ ]:
catalog.bars() # no bars yet

### Define BarType and Instrument

We can't infer this from our data, we just "know" what this is based on the url we queried. 

We may be able to download the _actual_ instrument definition from a Nautilus Adapter, but often when testing it's easier to just generate a dummy instrument for backtests. 

In [ ]:
from nautilus_trader.model.identifiers import Venue

# All instruments belong to a venue
FTX = Venue("FTX")

In [ ]:
from decimal import Decimal
from nautilus_trader.model.currency import Currency
from nautilus_trader.model.identifiers import Symbol
from nautilus_trader.model.instruments.currency_pair import CurrencyPair
from nautilus_trader.model.currencies import BTC
from nautilus_trader.model.currencies import ETH
from nautilus_trader.model.currencies import USD
from nautilus_trader.model.objects import Money
from nautilus_trader.model.objects import Price
from nautilus_trader.model.objects import Quantity
from nautilus_trader.model.identifiers import InstrumentId

# Define a helper function to create the instrument, incase we want to create muleiple.

def create_instrument(base: Currency, quote: Currency, venue: Venue):
    symbol = Symbol(f"{base.code}{quote.code}")
    instrument_id=InstrumentId(symbol=symbol, venue=venue)
    
    return CurrencyPair(
        instrument_id=instrument_id,
        native_symbol=symbol,
        base_currency=base,
        quote_currency=quote,
        price_precision=2,
        size_precision=6,
        price_increment=Price(1e-02, precision=2),
        size_increment=Quantity(1e-06, precision=6),
        lot_size=None,
        max_quantity=Quantity(9000, precision=6),
        min_quantity=Quantity(1e-06, precision=6),
        max_notional=None,
        min_notional=Money(10.00000000, quote),
        max_price=Price(1000000, precision=2),
        min_price=Price(0.01, precision=2),
        margin_init=Decimal(0),
        margin_maint=Decimal(0),
        maker_fee=Decimal("0.001"),
        taker_fee=Decimal("0.001"),
        ts_event=0,
        ts_init=0,
    )

instrument = create_instrument(base=BTC, quote=USD, venue=FTX)

In [ ]:
from nautilus_trader.model.data.bar import BarType
# Define the bar type for this file, we can't infer it from the actual data; we "just know" because of the url we downloaded it from.

bar_type = BarType.from_str(f"{instrument.id.value}-1-TICK-LAST-EXTERNAL") # Short hand for defining a bar from a string

### Write the parsing function

We need to convert each line to a nautilus object.

In [ ]:
import pandas as pd
from nautilus_trader.model.data.bar import Bar, BarType
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.core.datetime import dt_to_unix_nanos


def parse_ftx_candle_line(line, bar_type: BarType, instrument):
    ts_init = dt_to_unix_nanos(pd.Timestamp(line['startTime']))
    return Bar(
        bar_type=bar_type,
        open=Price(line['open'], precision=instrument.price_precision),
        high=Price(line['high'], precision=instrument.price_precision),
        low=Price(line['low'], precision=instrument.price_precision),
        close=Price(line['close'], precision=instrument.price_precision),
        volume=Quantity(line['volume'], precision=instrument.size_precision),
        ts_init=ts_init,
        ts_event=ts_init,
    )

In [ ]:
# Confirm its working
parse_ftx_candle_line(line=raw['result'][0], bar_type=bar_type, instrument=instrument)

In [ ]:
from nautilus_trader.persistence.external.core import write_objects

In [ ]:
bars = [
    parse_ftx_candle_line(line=line, bar_type=bar_type, instrument=instrument)
    for line in raw['result']
]

len(bars)

In [ ]:
# `write_objects` allows writing arbitrary objects to the catalog

write_objects(catalog=catalog, chunk=bars)

In [ ]:
catalog.bars().head()